In [3]:
# this code is intended to be a minimum viable reproduction of the process for adding a bus stop in Genet to try and understand why it is throwing a mobsim error.

from genet import Route, Service, Stop, read_matsim
import pandas as pd
import os

# baseline 2040 network
path_to_matsim_network = "/mnt/c/_BERTIE_data/2_2040_do_minimum_40WFH/"
# chosen output location
output_path = "/mnt/c/_BERTIE_data/bug_test"


In [4]:
# read the network:

network = os.path.join(path_to_matsim_network, "output_network.xml")
schedule = os.path.join(path_to_matsim_network, "output_transitSchedule.xml") 
vehicles = os.path.join(path_to_matsim_network, 'output_transitVehicles.xml')
n = read_matsim(
    path_to_network=network, epsg="epsg:27700", path_to_schedule=schedule, path_to_vehicles=vehicles
)

In [5]:
# check the network:

n

<Network instance at 136046531779024: with 
graph: MultiDiGraph with 447655 nodes and 985382 edges and 
schedule Schedule:
Number of services: 1351
Number of routes: 4687
Number of stops: 20219

In [6]:
# rep = n.generate_validation_report() 

# just checking everything is okay before we take things any further - if there is a speed error, then we know we cannot trust the validation reports!

# ok, no speed error - so it's definitely something we did.

In [7]:
# bus stop IDs have changed since older renditions of BERTIE
# fixed it. was 5177495089997857897; dug out relevant stop id from schedule xml
# appears stop ids have changed in the newer versions of BERTIE

n.schedule.stop('390060143').print()

Stop ID: 390060143
Projection: epsg:27700
Lat, Lon: 52.150799, 1.60107101
linkRefId: 5177495089997857897_5177495089997857897


In [8]:
# two relevant stops for MVP - they are quite far away, sorry!

market_town_stops = {
    'Aldeburgh': '390060144',
    'Woodbridge': '390060607'
}

In [9]:
# adding a new route:

route = Route(
    route_short_name="new_route",
    mode="bus",
    # five services between 7am and 8am
    headway_spec={("07:00:00", "08:00:00"): 15},
    arrival_offsets=["00:00:00","00:04:00",],
    departure_offsets=["00:00:00","00:04:00",],
    id="new_route",
    # do not need exact route; route using shutil later.
    #    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
    await_departure=[True, True],
    # two stops, in woodbridge and aldeburgh
    stops=[
        n.schedule.stop("390060144"),
        n.schedule.stop("390060607"),
    ],
)

# vehs created automatically, see below

In [10]:
# check everything looks normal:

# route.trips

In [11]:
# turn route into service:

new_service = Service(id="new_service", routes=[route])
new_service.print()

Service ID: new_service
Name: new_route
Number of routes: 1
Number of stops: 2


In [12]:
# check the number of vehicles in the schedule:

len(n.schedule.vehicles)

52124

In [ ]:
# add the service to the schedule:

n.schedule.add_service(Service(id="new_service", routes=[route]))

# whether vehicles are generated or not, makes no difference to the error:

n.schedule.generate_vehicles()

2026-01-30 10:23:50,327 - Added Services with IDs `['new_service']` and Routes: [['new_route']]


In [14]:
# check the number of vehicles in the schedule:

len(n.schedule.vehicles)

# good, so five instances of that service were added!

52129

In [15]:
# from genet reference docs - says no problem        

n.schedule.validate_vehicle_definitions()

True

In [16]:
# check schedule has been changed correctly.

n.schedule.change_log().tail()

# ok, it's in the frame, just have to hope it runs properly in the model...

# may need to add 'force=True' to end of changes at schedule.add_route - but not convinced this is the error at the moment.

,timestamp,change_event,object_type,old_id,new_id,old_attributes,new_attributes,diff
0,2026-01-30 10:23:50,add,service,None,new_service,None,"{'id': 'new_service', 'name': 'new_route'}","[(add, , [('id', 'new_service'), ('name', 'new..."


In [17]:
# this output is also correct

n.schedule.vehicle_types

{'bus': {'capacity': {'seats': '7',
   'standingRoom': '0',
   'length': {'meter': '18.0'},
   'width': {'meter': '2.5'},
   'costInformation': {},
   'passengerCarEquivalents': {'pce': '0.28'},
   'networkMode': {'networkMode': 'car'},
   'flowEfficiencyFactor': {'factor': '1.0'}},
  'attribute': {'name': 'egressTimeInSecondsPerPerson',
   'class': 'java.lang.Double'},
  'attributes': {}},
 'ferry': {'capacity': {'seats': '25',
   'standingRoom': '0',
   'length': {'meter': '50.0'},
   'width': {'meter': '6.0'},
   'costInformation': {},
   'passengerCarEquivalents': {'pce': '0.71'},
   'networkMode': {'networkMode': 'car'},
   'flowEfficiencyFactor': {'factor': '1.0'}},
  'attribute': {'name': 'egressTimeInSecondsPerPerson',
   'class': 'java.lang.Double'},
  'attributes': {}},
 'rail': {'capacity': {'seats': '100',
   'standingRoom': '0',
   'length': {'meter': '200.0'},
   'width': {'meter': '2.8'},
   'costInformation': {},
   'passengerCarEquivalents': {'pce': '2.71'},
   'networ

In [18]:
# this should be empty - in other words, the new service has yet to be connected to the network:

n.schedule["new_service"].route("new_route").network_links

[]

In [19]:
# make sure stops are ordered:

n.schedule["new_service"].route("new_route").ordered_stops

['390060144', '390060607']

In [20]:
# now that the service is in the schedule - we can route it:

import shutil
# I installed CBC to make this work: https://github.com/coin-or/Cbc

if shutil.which("cbc"):
    n.route_service("new_service")
else:
    print("Cannot route service without a solver installed")

2026-01-30 10:29:51,287 - Routing Service new_service with modes = {'bus'}
2026-01-30 10:29:59,345 - Building Maximum Stable Set for PT graph with 2 stops and 1 edges
2026-01-30 10:30:00,622 - Passing problem to solver
2026-01-30 10:30:00,623 - Initializing ordered Set vertices with a fundamentally unordered data source (type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
2026-01-30 10:30:00,625 - Passing problem to solver
2026-01-30 10:30:01,710 - Stop ID changes detected for Routes: {'new_route'}
2026-01-30 10:30:01,797 - Changed Route attributes for 1 routes
2026-01-30 10:30:01,846 - Changed Link attributes for 210 links


In [21]:
# this now yields a long list - showing that yes, the bus route has been routed

n.schedule["new_service"].route("new_route").network_links

['290888',
 '274265',
 '123093',
 '544081',
 '535399',
 '294036',
 '391093',
 '434409',
 '5177494990721463477_5177494990762671725',
 '5177494990762671725_5177494990769923335',
 '158804',
 '229088',
 '5177494990866370553_5177494990244302015',
 '44173',
 '331842',
 '5177494989857032495_5177494992747871183',
 '458801',
 '376490',
 '533562',
 '5177494986463186391_5177494986466075127',
 '533708',
 '13008',
 '5177494986349343671_5177494986342750999',
 '73945',
 '5177494983750520715_5177494983756520919',
 '474550',
 '573289',
 '5177494793405396543_5177494794049255669',
 '5177494794049255669_5177494794044240991',
 '564913',
 '380869',
 '5177494792452276933_5177494794979525915',
 '542406',
 '5177494799203938617_5177494799113549529',
 '5177494799113549529_5177494799089164515',
 '560421',
 '402434',
 '197259',
 '570715',
 '357776',
 '410783',
 '317408',
 '5177316754277025517_5177316754328059643',
 '479752',
 '560065',
 '597455',
 '616727',
 '19103',
 '85463',
 '352544',
 '187403',
 '399630',
 '26

In [22]:
# this is the same output but still looks good

n.schedule["new_service"].route("new_route").ordered_stops

['390060144.link:290888', '390060607.link:571846']

In [32]:
# provided the above suggests new_service is valid, write to disk.

n.write_to_matsim(output_path)  

2026-01-30 10:30:52,108 - Writing /mnt/c/_BERTIE_data/bug_test/network.xml
2026-01-30 10:36:47,573 - Writing /mnt/c/_BERTIE_data/bug_test/schedule.xml
2026-01-30 10:38:49,137 - Writing /mnt/c/_BERTIE_data/bug_test/vehicles.xml


Below is just troubleshooting - can skip.

In [23]:
# vehicle definition check - from DeepWiki:

print(f"Total vehicles: {len(n.schedule.vehicles)}")  
print(f"Vehicle types: {n.schedule.vehicle_types.keys()}")  
  
print(new_service.routes)

# Check vehicles for your new service specifically  
# new_service = n.schedule["new_service"]  
# for r in new_service.routes:  
#     print(f"Route {route.id} vehicles: {r.vehicles}")  
  
# # Validate vehicle definitions  
n.schedule.validate_vehicle_definitions()  
missing_info = n.schedule.get_missing_vehicle_information()  
print(f"Missing vehicle info: {missing_info}")

Total vehicles: 52129
Vehicle types: dict_keys(['bus', 'ferry', 'rail', 'subway', 'tram'])
<bound method Service.routes of <Service instance at 136041839630848: with 1 routes>>
Missing vehicle info: {'missing_vehicle_types': set(), 'vehicles_affected': {}}


In [24]:
# service routing check - from DeepWiki:

# Check if the service was properly routed  
route = n.schedule["new_service"].route("new_route")  
print(f"Network links: {len(route.network_links) if route.network_links else 0}")  
print(f"Ordered stops: {route.ordered_stops}")  
  
# Verify all stops have link references  
for stop in route.stops():  
    has_link = hasattr(stop, 'linkRefId')  
    print(f"Stop {stop.id}: has linkRefId = {has_link}")

Network links: 210
Ordered stops: ['390060144.link:290888', '390060607.link:571846']
Stop 390060144.link:290888: has linkRefId = True
Stop 390060607.link:571846: has linkRefId = True


In [25]:
# verify stop facilities - from DeepWiki:

for stop_id in ["390060144", "390060607"]:  
    stop = n.schedule.stop(stop_id)  
    print(f"Stop {stop_id}:")  
    print(f"  ID: {stop.id}")  
    print(f"  Link ref: {getattr(stop, 'linkRefId', 'MISSING')}")  
    print(f"  Attributes: {getattr(stop, 'attributes', {})}")

Stop 390060144:
  ID: 390060144
  Link ref: 5177495089997857897_5177495089997857897
  Attributes: {}
Stop 390060607:
  ID: 390060607
  Link ref: 5177341339775879099_5177341339775879099
  Attributes: {}


In [26]:
# check structural questions - from DeepWiki:

# Check for common structural issues  
print(f"Services: {n.schedule.service_ids()}")  
print(f"Routes per service: {n.schedule.service_to_route_map()}")  
  
# Ensure your new service is properly structured  
new_service = n.schedule["new_service"]  
print(f"New service routes: {new_service.route_ids()}")  

for route_id in new_service.route_ids():  
    route = new_service.route(route_id)  
    print(f"Route {route_id}: {len(list(route.stops()))} stops, {len(route.vehicles())} vehicles")

Services: ['990', '60830', '40226', '14037', '44148', '69108', '6658', '38434', '66603', '66515', '61194', '32033', '38364', '38323', '14324', '59352', '11878', '26941', '66110', '15001', '68544', '44145', '26927', '62879', '51153', '33822', '60772', '56170', '50112', '67254', '19696', '20626', '15863', '15392', '18853', '62937', '53009', '33036', '62911', '60825', '20190613_0823_50001548', '39161', '70755', '19878', '5007', '65293', '20190613_0823_50000372', '33794', '55072', '60871', '18064', '13429', '64736', '62903', '58947', '53455', '62938', '13721', '67075', '66103', '6741', '18559', '46109', '37129', '14590', '45664', '40227', '6235', '31924', '37721', '29874', '55095', '59658', '6724', '58605', '50173', '4968', '56160', '63136', '4946', '30706', '15204', '9483', '63999', '52725', '4943', '16730', '51041', '62929', '56157', '68555', '4947', '62655', '50245', '69118', '8632', '35347', '53713', '66214', '13002', '11432', '12430', '55932', '20190613_0823_50001550', '9765', '62797'

In [27]:
# Check the raw graph structure - from DeepWiki:
print(f"Number of routes: {len(n.schedule.route_ids())}")  
print(f"Number of services: {len(n.schedule.service_ids())}")  

Number of routes: 4688
Number of services: 1352


In [29]:
# check if route specifically is valid - it says it is

route = new_service.route("new_route")
is_valid, invalid_stages = route.is_valid_route(return_reason=True)
print(f"Valid: {is_valid}, Failed stages: {invalid_stages}")

Valid: True, Failed stages: []


In [ ]:
# full validation report - not recommended as takes a honking long time

# rep = n.generate_validation_report()

In [ ]:
# further genet checks via DeepWiki:

# # Check if all stops have consistent coordinate projections  
# has_uniform_projection = n.schedule.has_uniformly_projected_stops()  
# print(f"Uniform projections: {has_uniform_projection}")  
  
# # Get unique projections to identify inconsistencies  
# unique_projections = n.schedule.unique_stop_projections()  
# print(f"Unique projections: {unique_projections}")  
  
# # Check for conflicting stop data  
# stops_without_data, stops_with_conflicting_data = n.schedule._compare_stops_data(graph_reference)  
# print(f"Stops without data: {stops_without_data}")  
# print(f"Stops with conflicting data: {stops_with_conflicting_data}")

In [ ]:
# reindexing - not currently necessary, but available: 

# n.schedule["new_service"].reindex(new_id="more_appropriate_id")